In [1]:
from irev.data import Database
import pandas as pd

In [2]:
import torch
import torch.nn as nn

In [3]:
bd = Database.Database(
    ".local/data/amazon_review_dataset.csv",
    ".local/data",
    mode="init"
    )

================== Saved Finished ==================
Train Size: 2420
Test Size: 1513
Val Size: 1513


In [4]:
bd.train.head()

,user_id,item_id,rating,text,timestamp
1361,190,25,3,"[ugh, way, larg, man]",1495670400
842,125,25,5,"[super, comfi]",1506470400
1453,202,13,4,"[lightweight, comfort, support, happi, bought,...",1493683200
2938,388,25,5,[comfort],1526428800
1705,233,25,5,"[inch, extra, space, tip, big, toe, front, sho...",1490227200


In [5]:
# Criar um tfidf para cada sentença
# Criar um documento concatenando todos os tfidfs

# Cria as embeddings para cada sentença
# Cria as embeddings para cada documento

In [6]:
a = "Primeira Pergunta Dia"
b = "Segunda Pergunta Dia"
c = "Terceira Pergunta Dia"
d = "Quarta Pergunta Dia"
# Preciso construir o vocabulario e o tamanho dele

### TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
lista = [a,b,c,d]

In [9]:
vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = vectorizer.fit_transform(lista)

In [10]:
print(tfidf_matrix.toarray()) # Transformando uma matriz csr em uma matriz densa

[[0.41988018 0.41988018 0.8046125  0.         0.         0.        ]
 [0.41988018 0.41988018 0.         0.         0.8046125  0.        ]
 [0.41988018 0.41988018 0.         0.         0.         0.8046125 ]
 [0.41988018 0.41988018 0.         0.8046125  0.         0.        ]]


In [11]:
vectorizer.vocabulary_

{'primeira': 2,
 'pergunta': 1,
 'dia': 0,
 'segunda': 4,
 'terceira': 5,
 'quarta': 3}

In [12]:
vectorizer.transform(lista)

<4x6 sparse matrix of type '<class 'numpy.float64'>'
	with 12 stored elements in Compressed Sparse Row format>

### Word2Vec

In [13]:
corpus = [text.split() for text in lista]
corpus

[['Primeira', 'Pergunta', 'Dia'],
 ['Segunda', 'Pergunta', 'Dia'],
 ['Terceira', 'Pergunta', 'Dia'],
 ['Quarta', 'Pergunta', 'Dia']]

In [14]:
from gensim.models import Word2Vec

In [15]:
model = Word2Vec(corpus, window=2, min_count=1)

In [16]:
import numpy as np

In [17]:
w2v_matrix = []

for text in corpus:
    new_text = []
    for word in text:
        new_text.append(model.wv[word])
    new_text = np.mean(np.array(new_text), axis=0)
    w2v_matrix.append(new_text)

In [18]:
w2v_matrix # Matriz de embeddings senteces

[array([-5.9611327e-03,  2.0107769e-03,  3.1398963e-03,  1.8107076e-03,
        -3.7547245e-03, -4.8984010e-03,  3.9989650e-03,  6.2414468e-03,
        -4.7847242e-03, -5.6033209e-03,  6.5695652e-04, -4.0630880e-03,
        -5.7956376e-03,  1.3517576e-03, -1.4170833e-03, -9.8439632e-04,
         1.2126504e-03,  9.5056230e-04, -4.8061688e-03, -3.9058737e-03,
         2.3438130e-04, -6.1323248e-05,  7.7663106e-03, -4.6828960e-04,
         1.9698052e-03, -1.3859267e-03, -9.2022697e-04,  1.9867951e-04,
        -2.5949243e-03,  3.0999156e-03,  2.7332294e-03, -4.6851826e-03,
         1.4738790e-03, -7.0516546e-03,  5.2100333e-04,  3.3708399e-03,
         6.9806068e-03,  1.7458359e-03,  4.8815743e-03,  4.2558866e-04,
         2.6155666e-03, -3.1128787e-03, -8.3952798e-03, -2.3376558e-03,
        -1.3045006e-03,  1.4606771e-03, -7.7175052e-04,  5.1859678e-03,
         1.9318344e-03,  3.1327477e-03,  8.0135092e-04, -4.4578295e-03,
         1.1710655e-03,  4.2766123e-03, -6.7481305e-04,  2.12195

### Embedding Layer

In [84]:
import torch
import torch.nn as nn

In [85]:
emb = nn.Embedding(6, 10)

In [86]:
max_seq_length = 5 # Número máximo de palavras em uma sentença
embedding_size = 10 # Tamanho do vetor a ser gerado em uma embedding

In [87]:
# Criar um dicionário de palavras e seus índices correspondentes:

word2idx = {"<PAD>": 0, "<UNK>": 1}  # adicionando tokens especiais para padding e desconhecido
for sentence in corpus:
    for word in sentence:
        if word not in word2idx:
            word2idx[word] = len(word2idx)
            
word2idx

{'<PAD>': 0,
 '<UNK>': 1,
 'Primeira': 2,
 'Pergunta': 3,
 'Dia': 4,
 'Segunda': 5,
 'Terceira': 6,
 'Quarta': 7}

In [88]:
# Criar uma lista de indices para cada sentença
# Se a sentença não tiver a palavra, preencher com zeros a direita até
# todas as listas terem o mesmo tamanho
indexed_sentences = []
for sentence in corpus:
    indexed_sentence = [word2idx.get(word, word2idx["<UNK>"]) for word in sentence]
    indexed_sentence += [word2idx["<PAD>"]] * (max_seq_length - len(indexed_sentence))
    indexed_sentences.append(indexed_sentence)
    
indexed_sentences

[[2, 3, 4, 0, 0], [5, 3, 4, 0, 0], [6, 3, 4, 0, 0], [7, 3, 4, 0, 0]]

In [89]:
indexed_sentences_tensor = torch.tensor(indexed_sentences, dtype=torch.long)

In [90]:
# (tamanho do vocab, tamanho da emb de saida, index referente ao padding)
embedding_layer = nn.Embedding(len(word2idx), embedding_size, padding_idx=0)

In [91]:
embedded_sentences = embedding_layer(indexed_sentences_tensor)
embedded_sentences

tensor([[[-1.6438,  0.9252, -1.5696, -0.4842, -0.4853,  0.4907,  0.9502,
           0.0519, -0.9888, -1.2807],
         [ 0.8546, -0.6612, -0.8398, -0.6380, -0.6011,  0.6486, -0.7704,
          -0.6828,  0.2890,  0.7763],
         [ 1.2898, -0.5775,  0.1939, -1.0228, -0.7897, -1.3141, -2.4340,
          -0.1878, -1.0832, -0.7687],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000]],

        [[-0.1566,  1.4703,  0.0842,  0.0208,  1.0347, -0.0311,  0.2105,
          -1.0327, -1.1158, -0.5674],
         [ 0.8546, -0.6612, -0.8398, -0.6380, -0.6011,  0.6486, -0.7704,
          -0.6828,  0.2890,  0.7763],
         [ 1.2898, -0.5775,  0.1939, -1.0228, -0.7897, -1.3141, -2.4340,
          -0.1878, -1.0832, -0.7687],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],

In [92]:
import torch.nn.functional as F

In [93]:
# (batch_size, num_palavras, embedding_size)
embedded_sentences.shape
# num_palavras - Número total de palavras por sentença
#

torch.Size([4, 5, 10])

**batch_size** representa o número de exemplos (amostras) de treinamento que serão usados em cada iteração (ou batch) do treinamento da rede neural. Por exemplo, se você tiver 1000 exemplos de treinamento e definir um batch_size de 32, a rede neural processará 32 exemplos de treinamento de cada vez e levará 1000/32 iterações para concluir uma época de treinamento.

**num_palavras** representa o número de palavras em cada exemplo (amostra) de treinamento. Em tarefas de NLP, cada exemplo geralmente é uma sentença, e o **num_palavras** é o número de palavras nessa sentença. Por exemplo, a sentença "O gato está em cima da mesa" tem num_palavras = 7. Em tarefas de NLP, é comum limitar o **num_palavras** a um valor fixo para simplificar o processamento da rede neural. Isso é conhecido como padding ou truncating e é útil para garantir que todas as entradas tenham o mesmo tamanho, o que facilita o processamento em lotes (batches) de exemplos.

In [94]:
batch_size = embedded_sentences.shape[0]
num_words = embedded_sentences.shape[1]
embedding_size = embedded_sentences.shape[2]

In [95]:
# Adicionando uma dimensão para os canais
embedded_sentences = embedded_sentences.unsqueeze(3)

In [96]:
embedded_sentences.shape

torch.Size([4, 5, 10, 1])

**in_channels** é o número de canais (ou mapas de características) na entrada da camada convolucional. Em outras palavras, é o número de "planos" de dados que a camada convolucional espera receber como entrada. Por exemplo, se a camada convolucional recebe uma imagem colorida como entrada, in_channels seria 3, correspondendo aos canais de vermelho, verde e azul.

**out_channels** é o número de filtros de convolução que a camada convolucional deve gerar. Cada filtro é aplicado a cada canal da entrada, produzindo um mapa de características (ou canal) na saída. Em outras palavras, out_channels é o número de "planos" de dados que a camada convolucional deve produzir na saída. Cada mapa de características representa um conjunto diferente de características extraídas da entrada, como bordas, texturas ou padrões.

In [97]:
in_channels = 1
out_channels = 32

##### Configurando a saída de uma camada de Embedding com a entrada da Convolucional:

in_channels = embedding_dim

In [98]:
conv = nn.Conv2d(5, 32, (5, 1))

In [99]:
# Aplicando a Conv
output = conv(embedded_sentences)
output.shape

torch.Size([4, 32, 6, 1])

In [100]:
# Aplicando a Conv com uma função de ativação
u_features = F.relu(conv(embedded_sentences))
u_features.shape

torch.Size([4, 32, 6, 1])

- 4 é o tamanho do lote (batch_size)
- 32 é o número de mapas de características de saída (out_channels)
- 6 é o tamanho da sequência de recursos (sequence length)
- 1 é o número de canais (in_channels)

In [101]:
# Aplicando uma max Polling
kernel_size = 2
stride = 1
maxpool = nn.MaxPool1d(kernel_size, stride)

In [102]:
u_features = maxpool(u_features.squeeze(3))

In [106]:
pooled_sequence_length = int((u_features.shape[2] - kernel_size) / stride) + 1
pooled_sequence_length

4

In [104]:
u_features.shape

torch.Size([4, 32, 5])

In [107]:
# achatar a saída da camada de max pooling em um tensor unidimensional
flattened_output = torch.flatten(u_features, start_dim=1)

In [108]:
# criar uma camada totalmente conectada com 128 neurônios de saída
fc_layer = nn.Linear(flattened_output.shape[1], 128)

In [109]:
# aplicar a camada totalmente conectada na saída achata da camada de max pooling
fc_output = fc_layer(flattened_output)

In [112]:
fc_output.shape

torch.Size([4, 128])

In [113]:
# Aplicando um dropout na camada linear
dropout = nn.Dropout(0.5)
u_features = dropout(fc_layer(flattened_output))

In [114]:
u_features

tensor([[-0.1299,  0.0000, -0.4641,  0.0140, -0.0000,  0.0901, -0.0000,  0.5400,
          0.0103,  0.6224, -0.1563, -0.3168,  0.0000, -0.0000, -0.1686,  0.4317,
          0.6308, -0.3785,  0.0000, -0.0068, -0.0000,  0.3081, -0.0000, -0.2691,
         -0.1095, -0.0000, -0.2887, -0.3964, -0.4780, -0.0000, -0.3941, -0.0842,
         -0.0000,  0.0000,  0.8834,  0.4940,  0.1893,  0.0000, -0.0000,  0.4273,
         -0.2535, -0.0000, -0.2096, -0.0000, -0.2713, -0.0000, -0.1481,  0.6080,
          0.1316,  0.0000,  0.7323,  0.0701, -0.0000,  0.0000,  1.0627,  0.0000,
         -0.0000, -0.0343,  0.0000, -0.0000, -0.4673,  0.0426,  0.0000, -0.0000,
         -0.5131, -0.0000, -0.2281, -0.0000,  0.0315,  0.0000, -0.1138, -0.2760,
          0.0000, -0.1161,  0.0000, -0.3690,  0.3451, -0.0313,  0.0000, -0.1170,
         -0.0000, -0.1781,  0.0870, -0.0000, -0.0000, -0.1570, -0.0000, -0.7528,
          0.0000, -0.0000, -1.0736,  0.2810,  0.0000,  0.5754,  0.0000,  0.3562,
         -0.0000, -0.0000, -

##### Uma rede convolucional com camada de embedding, convolução, max pooling e totalmente conectada pode ser usada para representar os comentários de um usuário em um espaço de características (ou features) e, em seguida, combinar essas representações em uma única representação que descreva o usuário como um todo.


- Camada de embedding: essa camada é responsável por transformar as palavras em vetores de tamanho fixo, que podem ser processados por camadas convolucionais. Cada palavra é mapeada para um vetor denso de tamanho fixo, de tal forma que palavras semanticamente similares são mapeadas para vetores que também são semanticamente similares. Esses vetores de palavras são usados como entrada para a próxima camada.

- Camadas convolucionais: essas camadas usam filtros para extrair informações relevantes das representações de palavras geradas pela camada de embedding. Cada filtro é uma matriz de pesos que é aplicada a um trecho da representação de entrada, e o resultado é uma ativação que representa a presença de uma determinada característica nessa parte da entrada. Diferentes filtros podem ser usados para detectar diferentes tipos de características, como padrões de palavras específicas ou n-gramas.

- Camadas de max pooling: essas camadas reduzem a dimensionalidade da saída das camadas convolucionais, selecionando o valor máximo em cada janela deslizante da representação da entrada. Isso é feito para identificar as características mais importantes em cada trecho da entrada.

- Camada totalmente conectada: a saída da camada de max pooling é achata para uma dimensão unidimensional e passada para uma camada totalmente conectada, que combina as características extraídas de todas as janelas em uma única representação do usuário. Essa camada pode ter várias unidades de saída, dependendo do tamanho do espaço de características desejado.

##### A saída final da rede é uma representação do usuário em um espaço de características. Essas características podem ser usadas para vários fins, como classificar usuários por interesse ou prever a probabilidade de que eles tomem uma determinada ação.